# Telco Churn Data Exploration

This notebook explores the telco customer churn dataset to understand the data structure, identify patterns, and gain insights for feature engineering and modeling.

## Import Required Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Load and Examine Data

In [6]:
# Load the dataset
# Note: Update the file path to your actual dataset location
try:
    df = pd.read_csv('data/raw/Telco-Customer-Churn.csv')
    print("✅ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print("❌ Dataset not found. Please ensure the data file is in the correct location.")
    print("Expected location: data/raw/Telco-Customer-Churn.csv")
    # Create a sample dataset for demonstration
    df = pd.DataFrame()
    print("Creating sample structure for demonstration...")

❌ Dataset not found. Please ensure the data file is in the correct location.
Expected location: data/raw/Telco-Customer-Churn.csv
Creating sample structure for demonstration...
